In [9]:
import tarfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

with tarfile.open("aclImdb_v1.tar.gz", "r:gz") as tar:
    tar.extractall()

FileNotFoundError: [Errno 2] No such file or directory: 'aclImdb_v1.tar.gz'